# Installation


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

# Upscale images

In [ ]:
#@title Create model
from RealESRGAN import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "2" #@param ["2", "4", "8"] {allow-input: false}

model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')

## OPTION: move files and fix filenames

In [ ]:
# !cp -r /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/*.png /content/drive/MyDrive/AI/realesrgan_upscaling/input/

In [ ]:
# %cd /content/drive/MyDrive/AI/realesrgan_upscaling/input/

In [ ]:
# %%sh
# for file in *-*.png
#  do
#    newName="${file#*-}"
#    mv -- "${file}" "${newName}"
#  done

## upscale

In [ ]:
!mkdir /tmp/skrek_upscale_2x

In [ ]:
!cp -r /project/input_imgs/complete_10_fps_generation /tmp/skrek_upscale_2x

In [ ]:
!ls /tmp/skrek_upscale_2x/complete_10_fps_generation

In [ ]:
from glob import glob
input_path = "/tmp/skrek_upscale_2x/"
filename_list = sorted(glob(input_path+"/*/*", recursive = True))
# filename_list

In [ ]:
#@title Upload and upscale images or .tar archives
import os
# from google.colab import files
import shutil
from io import BytesIO
import io
import tarfile

upload_folder = '/tmp/skrek_upscale_2x/inputs'
result_folder = '/tmp/skrek_upscale_2x/results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = io.BufferedReader(buff)
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(buff.getvalue())
    return img_tar_info, fp

def process_tar(path_to_tar):
    processing_tar = tarfile.open(path_to_tar, mode='r')
    result_tar_path = os.path.join(result_folder+'/', os.path.basename(path_to_tar))
    save_tar = tarfile.open(result_tar_path, 'w')

    for c, member in enumerate(processing_tar):
        print(f'{c}, processing {member.name}')

        if not member.name.endswith(IMAGE_FORMATS):
            continue

        try:
            img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
            img_lr = Image.open(img_bytes, mode='r').convert('RGB')
        except Exception as err:
            print(f'Unable to open file {member.name}, skipping')
            continue

        img_sr = model.predict(np.array(img_lr))
        # adding to save_tar
        img_tar_info, fp = image_to_tar_format(img_sr, member.name)
        save_tar.addfile(img_tar_info, fp)

    processing_tar.close()
    save_tar.close()    
    print(f'Finished! Archive saved to {result_tar_path}')

def process_input(filename):
    if tarfile.is_tarfile(filename):
        process_tar(filename)
    else:
        result_image_path = os.path.join(result_folder+'/', os.path.basename(filename))
        image = Image.open(filename).convert('RGB')
        sr_image = model.predict(np.array(image))
        sr_image.save(result_image_path)
        print(f'Finished! Image saved to {result_image_path}')

# upload files
# uploaded = files.upload()
for filename in filename_list:
    print('Processing:', filename)
    process_input(filename)

# # upload files
# uploaded = files.upload()
# for filename in uploaded.keys():
#     print('Processing:', filename)
#     process_input(filename)

In [ ]:
# output_path = "/content/drive/MyDrive/AI/realesrgan_upscaling/output/"

In [ ]:
# !mv /content/results/* $output_path

In [ ]:
!mv $result_folder /project/output_imgs

# Convert output PNG files to JPG

In [ ]:
from PIL import Image
from tqdm.notebook import tqdm

def convert_png_to_jpg(filename, savename):
  im = Image.open(filename)
  rgb_im = im.convert('RGB')
  rgb_im.save(savename)


In [ ]:
input_png_dir = '/content/drive/MyDrive/AI/realesrgan_upscaling/output'
output_png_dir = '/content/drive/MyDrive/AI/realesrgan_upscaling/JPG'
images = []

for file_name in tqdm(sorted(os.listdir(input_png_dir))):
  # filter on PNG files
    # if file_name.endswith('.png'):
      # get the complete input/output paths
      file_path_input = os.path.join(input_png_dir, file_name)
      file_path_ouput = os.path.join(output_png_dir, file_name)
      file_path_ouput = file_path_ouput.replace('.png', '.jpg')

      convert_png_to_jpg(file_path_input, file_path_ouput)
